In [ ]:
import os
import pickle
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
import os
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from torchvision import transforms
import random
import torch
import torch.optim as optim
import torch.nn as nn
import pydicom
import glob
import cv2
import tqdm
from scipy.ndimage.morphology import binary_dilation
import torch.nn as nn 
import torch.nn.functional as F

In [ ]:
num_workers = 8
batch_size = 1
input_dir = '../../input'

device = 'cuda'

In [ ]:
test_df = pd.read_csv(os.path.join(input_dir, 'sample_submission.csv'))
test_df['ImagePath'] = test_df['ImageId'].apply(lambda x : os.path.join(input_dir, 'test', x) + '.dcm')
test_df[' EncodedPixels'] = [' -1']*test_df.shape[0]

In [ ]:
class SegmentationDataset(Dataset):

    def __init__(self,
                 df, 
                 tfms = None):

        self.df = df
        self.ids_list = self.df["ImageId"].unique()

        self.tfms = tfms

    def __len__(self):
        return len(self.ids_list)

    def __getitem__(self, idx):
        img_id = self.ids_list[idx]

        df_masks = self.df[self.df["ImageId"] == img_id]
        img_path = df_masks.iloc[0]["ImagePath"]

        img = pydicom.read_file(img_path).pixel_array
        segm = np.zeros_like(img)

        img = np.repeat(np.expand_dims(img, axis=2), 3, axis=2)

        if df_masks.iloc[0][' EncodedPixels'] != ' -1':
            for i in range(len(df_masks)):
                segm += rle2mask(df_masks.iloc[i][' EncodedPixels'], img.shape[0], img.shape[1]).astype(np.uint8).T

        segm = np.expand_dims(segm, axis=2)

        data = {"image": img, "mask": segm}

        augmented = self.tfms(**data)
        img, segm = augmented["image"], augmented["mask"]

        return transforms.ToTensor()(img), transforms.ToTensor()(segm)

In [ ]:
from albumentations import Resize, Compose, Normalize, ShiftScaleRotate

def get_segm_dl(img_size):
    tfms_val = Compose([
        Resize(img_size, img_size),
        Normalize(mean=(0.49, 0.49, 0.49), std=(0.235, 0.235, 0.235), max_pixel_value=255.0, always_apply=True, p=1.0)
    ], p=1.0)
    test_ds = SegmentationDataset(test_df, tfms = tfms_val)
    test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return test_dl

tfms_val_class = Compose([
    Resize(768, 768),
    Normalize(mean=(0.49, 0.49, 0.49), std=(0.235, 0.235, 0.235), max_pixel_value=255.0, always_apply=True, p=1.0)
], p=1.0)

tfms_val_tta_class = Compose([
    Resize(512, 512),
    ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, interpolation=1, border_mode=0, always_apply=False, p=1.0),
    Normalize(mean=(0.49, 0.49, 0.49), std=(0.235, 0.235, 0.235), max_pixel_value=255.0, always_apply=True, p=1.0)
], p=1.0)

#holdout_ds = SegmentationDataset(df, tfms = tfms_val)
#holdout_ds_class = SegmentationDataset(df, tfms = tfms_val_class)
#holdout_ds_class_tta = SegmentationDataset(df, tfms = tfms_val_tta_class)

test_ds_class = SegmentationDataset(test_df, tfms = tfms_val_class)
test_ds_class_tta = SegmentationDataset(test_df, tfms = tfms_val_tta_class)

#holdout_dl = torch.utils.data.DataLoader(holdout_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)
#holdout_dl_class = torch.utils.data.DataLoader(holdout_ds_class, batch_size=batch_size, shuffle=False, num_workers=num_workers)
#holdout_dl_class_tta = torch.utils.data.DataLoader(holdout_ds_class_tta, batch_size=batch_size, shuffle=False, num_workers=num_workers)

test_dl_class = torch.utils.data.DataLoader(test_ds_class, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_dl_class_tta = torch.utils.data.DataLoader(test_ds_class_tta, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
img_size = 128

tfms_val = Compose([
    Resize(img_size, img_size),
    Normalize(mean=(0.49, 0.49, 0.49), std=(0.235, 0.235, 0.235), max_pixel_value=255.0, always_apply=True, p=1.0)
], p=1.0)
test_ds = SegmentationDataset(test_df, tfms = tfms_val)

In [ ]:
class Fake_segmentation_models:
    class SENet154(nn.Module):
        def __init__(self, num_classes, pretrained=True, dropout=0.5):
            super(SENet154, self).__init__()
            self.encoder = pretrainedmodels.models.senet.senet154()

            self.encoder.dropout = nn.Dropout(dropout)
            self.encoder.avg_pool = nn.AdaptiveAvgPool2d(1)
            self.encoder.last_linear = nn.Linear(2048, num_classes)

        def forward(self, x): 
            x = self.encoder(x)

            return x

    class SEResNetx101_32(nn.Module):
        def __init__(self, num_classes, pretrained=True, dropout=0.5):
            super(SEResNetx101_32, self).__init__()
            self.encoder = pretrainedmodels.models.senet.se_resnext101_32x4d()

            self.encoder.dropout = nn.Dropout(dropout)
            self.encoder.avg_pool = nn.AdaptiveAvgPool2d(1)
            self.encoder.last_linear = nn.Linear(2048, num_classes)

        def forward(self, x): 
            x = self.encoder(x)

            return x

sys.modules["segmentation"] = None
sys.modules["segmentation.models"] = Fake_segmentation_models
#sys.modules["segmentation.models.SENet154"] = SENet154

    
class CBlendModel( nn.Module ):
    def __init__(self, models, ths):
        super(CBlendModel, self).__init__()
        self.models = models
        self.x = torch.nn.Parameter(torch.ones(len(self.models), requires_grad=True) / len(self.models))
        self.ths = ths

    def forward(self, inputs):
        n = len(self.models)
        output = 0
        
        for i in range(n):
            with torch.no_grad():
                self.models[i].eval()
                tmp = self.models[i](inputs)#.exp()
                #tmp = tmp/(1+tmp)

                tmp_flip = self.models[i](torch.flip(inputs, dims=(3,)))#.exp()
                #tmp_flip = tmp_flip/(1+tmp_flip)
                tmp = (tmp + tmp_flip)/2
                out = tmp * self.x[i]
                #out = (tmp > 1.0) * 1.0
                
            output += out  #/ self.x.sum()
    
        return output
    
class_model = torch.load("cblend_rex101_2x_senet154.pth", map_location="cpu")
class_model = class_model.to(device)

In [ ]:
for i in range(len(class_model.models)):
    class_model.models[i].to(device)

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

import re
import torch
from collections import OrderedDict

import torch.nn as nn
import torch
import torch.utils.model_zoo as model_zoo

class Fake_Module_resnetx:
    def conv3x3(in_planes, out_planes, stride=1):
        """3x3 convolution with padding"""
        return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                         padding=1, bias=False)

    class ConvBn2d(nn.Module):
        def __init__(self, in_channels, out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)):
            super(ConvBn2d, self).__init__()

            self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                                  bias=False)
            self.bn = nn.BatchNorm2d(out_channels)
            # self.bn = SynchronizedBatchNorm2d(out_channels)

        def forward(self, z):
            x = self.conv(z)
            x = self.bn(x)
            return x


    class ConvRelu(nn.Module):
        def __init__(self, in_, out):
            super().__init__()
            self.conv = conv3x3(in_, out)
            self.activation = nn.ReLU(inplace=True)

        def forward(self, x):
            x = self.conv(x)
            x = self.activation(x)
            return x


    class DecoderBlockV2(nn.Module):
        def __init__(self, in_channels, middle_channels, out_channels, is_deconv=True):
            super(DecoderBlockV2, self).__init__()
            self.in_channels = in_channels

            if is_deconv:
                """
                    Paramaters for Deconvolution were chosen to avoid artifacts, following
                    link https://distill.pub/2016/deconv-checkerboard/
                """

                self.block = nn.Sequential(
                    ConvRelu(in_channels, middle_channels),
                    nn.ConvTranspose2d(middle_channels, out_channels, kernel_size=4, stride=2,
                                       padding=1),
                    nn.ReLU(inplace=True))
            else:
                self.block = nn.Sequential(
                    nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                    ConvRelu(in_channels, middle_channels),
                    ConvRelu(middle_channels, out_channels))

        def forward(self, x):
            return self.block(x)


    class DecoderSEBlockV2(nn.Module):
        def __init__(self, in_channels, middle_channels, out_channels):
            super().__init__()
            self.in_channels = in_channels
            self.block = nn.Sequential(
                nn.Upsample(scale_factor=2, mode='bilinear'),
                ConvRelu(in_channels, middle_channels),
                ConvRelu(middle_channels, out_channels))
            # SEBlock(planes=out_channels, reduction=16))

        def forward(self, x):
            return self.block(x)


    class DecoderSEBlockV3(nn.Module):
        def __init__(self, in_channels, middle_channels, out_channels):
            super().__init__()
            self.in_channels = in_channels
            self.block = nn.Sequential(
                nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                ConvRelu(in_channels, middle_channels),
                ConvRelu(middle_channels, out_channels),
                SEBlock(planes=out_channels, reduction=16))

        def forward(self, x):
            return self.block(x)


    class SENeXt50(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=False, is_deconv=False):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = se_resnext50(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.conv1,
                    encoder.bn1,
                    encoder.relu,
                    self.pool),
                encoder.layer1,
                encoder.layer2,
                encoder.layer3,
                encoder.layer4])

            self.center = DecoderBlockV2(2048, num_filters * 8 * 2, num_filters * 8, is_deconv)

            self.decoder = nn.ModuleList([
                DecoderBlockV2(2048 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2, is_deconv),
                DecoderBlockV2(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2, is_deconv),
            ])

            self.dec1 = DecoderBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters, is_deconv)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())

            x = self.center(self.pool(x))

            for i, decoder in enumerate(self.decoder):
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

            x = self.dec1(x)
            x = self.dec0(x)
            x = self.final(x)

            return x


    class NeXt50(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=False, is_deconv=False):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = resnext50(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.conv1,
                    encoder.bn1,
                    encoder.relu,
                    self.pool),
                encoder.layer1,
                encoder.layer2,
                encoder.layer3,
                encoder.layer4])

            self.center = DecoderBlockV2(2048, num_filters * 8 * 2, num_filters * 8, is_deconv)

            self.decoder = nn.ModuleList([
                DecoderBlockV2(2048 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2, is_deconv),
                DecoderBlockV2(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2, is_deconv),
            ])

            self.dec1 = DecoderBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters, is_deconv)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())

            x = self.center(self.pool(x))

            for i, decoder in enumerate(self.decoder):
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

            x = self.dec1(x)
            x = self.dec0(x)
            x = self.final(x)

            return x


    class NeXt101(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=False, is_deconv=False):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = resnext101(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.conv1,
                    encoder.bn1,
                    encoder.relu,
                    self.pool),
                encoder.layer1,
                encoder.layer2,
                encoder.layer3,
                encoder.layer4])

            self.center = DecoderBlockV2(2048, num_filters * 8 * 2, num_filters * 8, is_deconv)

            self.decoder = nn.ModuleList([
                DecoderBlockV2(2048 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2, is_deconv),
                DecoderBlockV2(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2, is_deconv),
            ])

            self.dec1 = DecoderBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters, is_deconv)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())

            x = self.center(self.pool(x))

            for i, decoder in enumerate(self.decoder):
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

            x = self.dec1(x)
            x = self.dec0(x)
            x = self.final(x)

            return x


    class SENeXt50WithoutPooling(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=False, is_deconv=False):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = se_resnext50(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.conv1,
                    encoder.bn1,
                    encoder.relu),
                encoder.layer1,
                encoder.layer2,
                encoder.layer3,
                encoder.layer4])

            self.center = DecoderBlockV2(2048, num_filters * 8 * 2, num_filters * 8, is_deconv)

            self.decoder = nn.ModuleList([
                DecoderBlockV2(2048 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2, is_deconv),
                DecoderBlockV2(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2, is_deconv),
            ])

            self.dec1 = DecoderBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters, is_deconv)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=3)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())

            x = self.center(self.pool(x))

            for i, decoder in enumerate(self.decoder):
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

            x = self.dec1(x)
            x = self.dec0(x)
            x = self.final(x)

            return x


    class MultiSENeXt50(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=False, is_deconv=False):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = se_resnext50(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.conv1,
                    encoder.bn1,
                    encoder.relu,
                    self.pool),
                encoder.layer1,
                encoder.layer2,
                encoder.layer3,
                encoder.layer4])

            self.center = DecoderBlockV2(2048, num_filters * 8 * 2, num_filters * 8, is_deconv)

            self.decoder = nn.ModuleList([
                DecoderBlockV2(2048 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv),
                DecoderBlockV2(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2, is_deconv),
                DecoderBlockV2(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2, is_deconv)])

            self.avgpool = nn.AvgPool2d(7)
            self.fc = nn.Linear(2048, 1)
            self.dec1 = DecoderBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters, is_deconv)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())
            x_cls = self.avgpool(x)
            x_cls = x_cls.view(x_cls.size(0), -1)
            x_cls = self.fc(x_cls).view(x_cls.size(0))

            x = self.center(self.pool(x))
            for i, decoder in enumerate(self.decoder):
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

            x = self.dec1(x)
            x = self.dec0(x)
            x = self.final(x)
            return x, x_cls


    class MultiSESENeXt50(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=False):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = se_resnext50(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.conv1,
                    encoder.bn1,
                    encoder.relu,
                    self.pool),
                encoder.layer1,
                encoder.layer2,
                encoder.layer3,
                encoder.layer4])

            self.avgpool = nn.AvgPool2d(8)
            self.fc = nn.Linear(2048, 1)

            self.center = DecoderSEBlockV2(2048, num_filters * 8 * 2, num_filters * 8)

            self.decoder = nn.ModuleList([
                DecoderSEBlockV2(2048 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
                DecoderSEBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
                DecoderSEBlockV2(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2),
                DecoderSEBlockV2(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2),
            ])

            self.dec1 = DecoderSEBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())

            # x_cls = self.avgpool(x)
            # x_cls = x_cls.view(x_cls.size(0), -1)
            # x_cls = self.fc(x_cls).view(x_cls.size(0))
            # print(x_cls.shape)

            x = self.center(self.pool(x))
            # print(x.shape)

            for i, decoder in enumerate(self.decoder):
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))
                # print(x.shape)

            x = self.dec1(x)
            # print(x.shape)
            x = self.dec0(x)
            # print(x.shape)
            x = self.final(x)
            # print(x.shape)
            # print('ok')

            return x  # , x_cls


    class MultiSESENeXt50_2(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=False):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = se_resnext50(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.conv1,
                    encoder.bn1,
                    encoder.relu,
                    self.pool),
                encoder.layer1,
                encoder.layer2,
                encoder.layer3,
                encoder.layer4])

            self.avgpool = nn.AvgPool2d(3)
            self.fc = nn.Linear(2048, 1)

            self.center = DecoderSEBlockV3(2048, num_filters * 8 * 2, num_filters * 8)

            self.decoder = nn.ModuleList([
                DecoderSEBlockV3(2048 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
                DecoderSEBlockV3(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
                DecoderSEBlockV3(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2),
                DecoderSEBlockV3(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2),
            ])

            self.dec1 = DecoderSEBlockV3(num_filters * 2 * 2, num_filters * 2 * 2, num_filters)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())

            # x_cls = self.avgpool(x)
            # x_cls = x_cls.view(x_cls.size(0), -1)
            # x_cls = self.fc(x_cls).view(x_cls.size(0))

            x = self.center(self.pool(x))

            for i, decoder in enumerate(self.decoder):
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

            x = self.dec1(x)
            x = self.dec0(x)
            x = self.final(x)

            return x  # , x_cls


    class MultiSESENeXt101(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=False):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = se_resnext101(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.conv1,
                    encoder.bn1,
                    encoder.relu,
                    self.pool),
                encoder.layer1,
                encoder.layer2,
                encoder.layer3,
                encoder.layer4])

            self.avgpool = nn.AvgPool2d(3)
            self.fc = nn.Linear(2048, 1)

            self.center = DecoderSEBlockV2(2048, num_filters * 8 * 2, num_filters * 8)

            self.decoder = nn.ModuleList([
                DecoderSEBlockV2(2048 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
                DecoderSEBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
                DecoderSEBlockV2(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2),
                DecoderSEBlockV2(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2),
            ])

            self.dec1 = DecoderSEBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())

            # x_cls = self.avgpool(x)
            # x_cls = x_cls.view(x_cls.size(0), -1)
            # x_cls = self.fc(x_cls).view(x_cls.size(0))

            x = self.center(self.pool(x))

            for i, decoder in enumerate(self.decoder):
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

            x = self.dec1(x)
            x = self.dec0(x)
            x = self.final(x)

            return x
        
class Fake_Module_sedensenet:
    model_urls = { 
    #    'densenet121': 'https://download.pytorch.org/models_zoo/densenet121-a639ec97.pth',
        'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
        'densenet169': 'https://download.pytorch.org/models_zoo/densenet169-b2777c0a.pth',
        'densenet201': 'https://download.pytorch.org/models_zoo/densenet201-c1103571.pth',
        'densenet161': 'https://download.pytorch.org/models_zoo/densenet161-8d451a50.pth',
    }


    class _DenseLayer(nn.Sequential):
        def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
            super(_DenseLayer, self).__init__()
            self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
            self.add_module('relu1', nn.ReLU(inplace=True)),
            self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                               growth_rate, kernel_size=1, stride=1, bias=False)),
            self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
            self.add_module('relu2', nn.ReLU(inplace=True)),
            self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                               kernel_size=3, stride=1, padding=1, bias=False)),
            self.drop_rate = drop_rate

        def forward(self, x):
            new_features = super(_DenseLayer, self).forward(x)
            if self.drop_rate > 0:
                new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
            return torch.cat([x, new_features], 1)


    class _DenseBlock(nn.Sequential):
        def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
            super(_DenseBlock, self).__init__()
            for i in range(num_layers):
                layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
                self.add_module('denselayer%d' % (i + 1), layer)


    class _Transition(nn.Sequential):
        def __init__(self, num_input_features, num_output_features):
            super(_Transition, self).__init__()
            self.add_module('norm', nn.BatchNorm2d(num_input_features))
            self.add_module('relu', nn.ReLU(inplace=True))
            self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                              kernel_size=1, stride=1, bias=False))
            self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


    class DenseNet(nn.Module):
        r"""Densenet-BC model class, based on
        `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

        Args:
            growth_rate (int) - how many filters to add each layer (`k` in paper)
            block_config (list of 4 ints) - how many layers in each pooling block
            num_init_features (int) - the number of filters to learn in the first convolution layer
            bn_size (int) - multiplicative factor for number of bottle neck layers
              (i.e. bn_size * k features in the bottleneck layer)
            drop_rate (float) - dropout rate after each dense layer
            num_classes (int) - number of classification classes
        """

        def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                     num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000, in_channels=3):

            super(DenseNet, self).__init__()

            # First convolution
            self.features = nn.Sequential(OrderedDict([
                ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
                ('norm0', nn.BatchNorm2d(num_init_features)),
                ('relu0', nn.ReLU(inplace=True)),
                ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
            ]))

            # Each denseblock
            num_features = num_init_features
            for i, num_layers in enumerate(block_config):
                block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                    bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
                self.features.add_module('denseblock%d' % (i + 1), block)
                num_features = num_features + num_layers * growth_rate
                if i != len(block_config) - 1:
                    trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                    self.features.add_module('transition%d' % (i + 1), trans)
                    num_features = num_features // 2

            # Final batch norm
            self.features.add_module('norm5', nn.BatchNorm2d(num_features))

            # Linear layer
            self.classifier = nn.Linear(num_features, num_classes)

            # Official init from torch repo.
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.constant_(m.bias, 0)

        def forward(self, x):
            x = self.features.conv0(x)
            x = self.features.norm0(x)
            x = self.features.relu0(x)
            print(x.size())
            x = self.features.pool0(x)

            x = self.features.denseblock1(x)
            print(x.size())
            x = self.features.transition1(x)

            x = self.features.denseblock2(x)
            print(x.size())
            x = self.features.transition2(x)

            x = self.features.denseblock3(x)
            print(x.size())
            x = self.features.transition3(x)

            x = self.features.denseblock4(x)
            x = self.features.norm5(x)
            print(x.size())

            out = F.relu(x, inplace=True)
            out = F.avg_pool2d(out, kernel_size=7, stride=1).view(x.size(0), -1)
            out = self.classifier(out)
            return out



    def densenet121(pretrained=True, **kwargs):
        r"""Densenet-121 model from
        `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_

        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16),
                         **kwargs)
        if pretrained:
            # '.'s are no longer allowed in module names, but pervious _DenseLayer
            # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
            # They are also in the checkpoints in model_urls. This pattern is used
            # to find such keys.
            pattern = re.compile(
                r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')
            state_dict = model_zoo.load_url(model_urls['densenet121'])
            for key in list(state_dict.keys()):
                res = pattern.match(key)
                if res:
                    new_key = res.group(1) + res.group(2)
                    state_dict[new_key] = state_dict[key]
                    del state_dict[key]
            model.load_state_dict(state_dict)
        return model


    class MultiSEDensenet121(nn.Module):
        def __init__(self, num_classes=1, num_filters=16, pretrained=True):
            super().__init__()
            self.num_classes = num_classes
            self.pool = nn.MaxPool2d(2, 2)
            encoder = densenet121(pretrained=pretrained)
            self.relu = nn.ReLU(inplace=True)

            self.encoder = nn.ModuleList([
                nn.Sequential(
                    encoder.features.conv0,
                    encoder.features.norm0,
                    encoder.features.relu0,
                ),

                nn.Sequential(
                    encoder.features.pool0,
                    encoder.features.denseblock1, ),
                nn.Sequential(
                    encoder.features.transition1,
                    encoder.features.denseblock2, ),
                nn.Sequential(
                    encoder.features.transition2,
                    encoder.features.denseblock3, ),
                nn.Sequential(
                    encoder.features.transition3,
                    encoder.features.denseblock4,
                    encoder.features.norm5),
            ])

            self.avgpool = nn.AvgPool2d(5)
            self.fc = nn.Linear(1024, 1)

            self.center = DecoderSEBlockV2(1024, num_filters * 8 * 2, num_filters * 8)

            self.decoder = nn.ModuleList([
                DecoderSEBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
                DecoderSEBlockV2(1024 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
                DecoderSEBlockV2(512 + num_filters * 8, num_filters * 4 * 2, num_filters * 2),
                DecoderSEBlockV2(256 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2),
            ])

            self.dec1 = DecoderSEBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters)
            self.dec0 = ConvRelu(num_filters, num_filters)
            self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

        def forward(self, x):
            encoder_results = []
            for stage in self.encoder:
                x = stage(x)
                encoder_results.append(x.clone())

            # print(x.size())
            #x_cls = self.avgpool(x)
            # print(x_cls.size())
            #x_cls = x_cls.view(x_cls.size(0), -1)
            # print(x_cls.size())

            #x_cls = self.fc(x_cls).view(x_cls.size(0))

            x = self.center(self.pool(x))

            for i, decoder in enumerate(self.decoder):
                # print(x.size(), encoder_results[-i - 1].size())
                x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

            x = self.dec1(x)
            x = self.dec0(x)
            x = self.final(x)
            return x #, x_cls

sys.modules['dtorch'] = None
sys.modules['dtorch.models'] = None
sys.modules['dtorch.models.resnetx'] = Fake_Module_resnetx
sys.modules['dtorch.models.sedensenet'] = Fake_Module_sedensenet

In [ ]:
from dtorch.models.sedensenet import MultiSEDensenet121
from dtorch.models.resnetx import DecoderBlockV2, DecoderSEBlockV2, ConvBn2d, ConvRelu, conv3x3
from dtorch.models.sedensenet import densenet121, DenseNet, _DenseBlock, _DenseLayer, _Transition, model_urls

In [ ]:
class BlendModel( nn.Module ):
    def __init__(self, models, ths):
        super(BlendModel, self).__init__()
        self.models = models
        self.x = torch.nn.Parameter(torch.ones(len(self.models), requires_grad=True) / len(self.models))
        self.ths = ths

    def forward(self, inputs):
        n = len(self.models)
        output = 0
        
        for i in range(n):
            with torch.no_grad():
                self.models[i].eval()
                tmp = self.models[i](inputs).exp()
                tmp = tmp/(1+tmp)
                tmp_flip = self.models[i](torch.flip(inputs, dims=(3,))).exp()
                tmp_flip = tmp_flip/(1+tmp_flip)
                if len(tmp_flip.shape) > 2:
                    tmp_flip = torch.flip(tmp_flip, dims=(3,))

                out = (tmp + tmp_flip)/2
                out = tmp * self.x[i]

        
        return output

model = torch.load("blend_dpn_dense_rex101_2x.pth")
model = model.to(device)

In [ ]:
def combine(pred_class, pred_seg, th_class, th_seg):
    pred_combined = []

    for i in range(len(pred_class)):
        if pred_class[i] <= th_class:
            pred_combined.append(np.zeros((1024, 1024)))
        else:
            pred_combined.append((pred_seg[i, :, :] > th_seg)*1.0)

    return np.stack(pred_combined, axis=0)

In [ ]:
from scipy.ndimage.morphology import binary_dilation

def drop_small_mask(predictions, th):
    for k in range(len(predictions)):
        mask = predictions[k]
        res = np.zeros_like(mask)

        ret, labels = cv2.connectedComponents(mask.astype(np.uint8), connectivity=4)
        for i in range(1, ret):
            if np.sum(labels == i) > th:
                res += (labels == i)
            #else:
            #    res += binary_dilation(labels == i, iterations=5)

        predictions[k] = res

    return predictions

In [ ]:
def predict(model, loader, device = torch.device("cuda" if torch.cuda.is_available() else "cpu")):

    result = []
    model.eval()
    
    with torch.no_grad():
        for datas in tqdm.tqdm(loader):
            inputs = datas[0]
            inputs = inputs.to(device)

            outputs = model(inputs)
            result.append(outputs.cpu().numpy())

    return np.concatenate(result, axis=0)

In [ ]:
th_seg = 0.35
th_class = 1.0
alpha = 0.6

In [ ]:
test_pred_class = predict(class_model, test_dl_class)
test_pred_class = np.squeeze(test_pred_class)

In [ ]:
test_dl = get_segm_dl(1536)

test_pred_1256 = predict(model, test_dl)
test_pred_1256 = np.array([np.squeeze(pred) for pred in test_pred_1256])

test_dl = get_segm_dl(1024)

test_pred = predict(model, test_dl)
test_pred = np.array([np.squeeze(pred) for pred in test_pred])

test_dl = get_segm_dl(768)

test_pred_768 = predict(model, test_dl)
test_pred_768 = np.array([np.squeeze(pred) for pred in test_pred_768])

In [ ]:
test_pred_res = np.stack([cv2.resize(test_pred_1256[i], (1024, 1024)) for i in range(len(test_pred_1256))])
test_pred_res1 = np.stack([cv2.resize(test_pred_768[i], (1024, 1024)) for i in range(len(test_pred_768))])

test_pred_tta = 0.5*test_pred + 0.3*test_pred_res + 0.2*test_pred_res1

In [ ]:
num = test_df.groupby("ImageId").apply(len)
leaked_ids = set(np.array(num[num > 1].index))

for i in range(len(test_ds_class.ids_list)):
    if test_ds_class.ids_list[i] in leaked_ids:
        test_pred_class[i] = 1000.

In [ ]:
#test_pred_class_combined = alpha * test_pred_class + (1 - alpha) * np.mean(np.stack(test_pred_class_tta), axis=0)

test_pred_combined = combine(test_pred_class, test_pred_tta, th_class, th_seg)
#test_pred_combined = drop_small_mask(test_pred_combined, 1000)

In [ ]:
np.sum(test_pred_class <= th_class)

In [ ]:
def subs(pred):
    if pred == '':
        return ' -1'
    return pred

In [ ]:
def mask2rle(img, width, height):
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1; 
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1; 
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1: 
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)


In [ ]:
img_ids = []
img_rles = []

for i in tqdm.tqdm(range(len(test_ds.ids_list))):
    
    img_ids.append(test_ds.ids_list[i])
    img_rles.append(mask2rle((test_pred_combined[i] > 0.5).T*255, 1024, 1024))

In [ ]:
submission_df = pd.DataFrame(np.array([img_ids, img_rles]).T, columns = ["ImageId", "EncodedPixels"])

submission_df["EncodedPixels"] = submission_df["EncodedPixels"].apply(subs)
submission_df[["ImageId", "EncodedPixels"]].to_csv("submission.csv", index=False)